In [1]:
import pandas as pd
import time, os, shutil
import re
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np  
from sklearn.cluster import KMeans  
from sklearn.cluster import DBSCAN
from nltk.parse import CoreNLPParser
from sklearn import decomposition as dc

#pos_tagger = CoreNLPParser(url='http://localhost:9000', tagtype='pos')

W0513 11:57:21.009261  8204 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [ ]:
data_source = 'output/sub-set/'
file_name = 'Fire HD'+'.csv'
df = pd.read_csv(data_source+file_name, sep="\t", error_bad_lines=False)
reviews = pd.DataFrame(df['text'])

In [ ]:
reviews['POS'] = reviews.text.apply(lambda x: pos_tagger.tag(x.split()))

In [ ]:
reviews.to_csv(r'/mnt/doc/School/USYD/Capstone/workspace/data_testing/output/sub-set/fireHD-tag.csv',sep='\t')

In [ ]:
reviews['POS'][0:10].tolist()

In [ ]:
def getMainWords(pos):
    result = '';
    for x in pos:
        if x[1][0:2] in ['NN','VB']:
            result= result+x[0]+' '
    if len(result)>0:
        result= result[0:len(result)-1]
    return result

reviews['main'] = list(map(getMainWords,reviews['POS']))
reviews = reviews.loc[reviews['main'].apply(len)>0,['text','main']] #filtering out sentences without any nouns or verbs

reviews = reviews.reset_index(drop=True)# reset index

In [2]:
#############################LOAD##################
# If load data with 'text' and 'main'
data_source = 'output/sub-set/'
file_name = 'fireHD-tag'+'.csv'
reviews = pd.read_csv(data_source+file_name, sep="\t", error_bad_lines=False).loc[:,['text','main']]
###################################################

In [3]:
# Load google Universal Sentence Encoder
module_dir ="downloads/encoder-DNA" #"downloads/encoder"
embed = hub.Module(module_dir)

Instructions for updating:
Colocations handled automatically by placer.


W0513 11:57:26.632224  8204 deprecation.py:323] From C:\Users\betty\Anaconda3\lib\site-packages\tensorflow\python\ops\control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [4]:
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    start_time=time.time()
    reviews['sentence_embedding'] = pd.Series(list(session.run(embed(list(reviews['text'])))))
    end_time1=time.time()
    reviews['words_embedding'] = pd.Series(list(session.run(embed(list(reviews['main'])))))
    end_time2=time.time()
    print('time1:',(end_time1-start_time),' time2:',(end_time2-end_time1))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0513 11:57:50.985045  8204 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0513 11:57:55.458122  8204 saver.py:1483] Saver not created because there are no variables in the graph to restore


time1: 5.202177286148071  time2: 4.444169521331787


In [97]:
words = ['library','librari']
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    re1 = session.run(embed(words))
    
np.inner(re1[0],re1[1])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0513 13:06:20.677272  8204 saver.py:1483] Saver not created because there are no variables in the graph to restore


0.24949102

In [5]:
reviews

,text,main,sentence_embedding,words_embedding
0,I find that is not as easy to use as my regula...,find is use Kindle Fire,"[0.074233115, 0.06930195, -0.031268492, 0.0549...","[0.07619723, 0.07710209, -0.02127747, 0.035578..."
1,When I am reading the kindle it freezes up (be...,am reading kindle freezes hold side,"[0.014984652, 0.0803408, -0.025731513, 0.01954...","[0.036976464, 0.06573325, -0.045748066, 0.0083..."
2,Also you have to get out of it to brighten or ...,have get brighten darken text are reading,"[-0.0010017281, 0.047674406, -0.015956037, 0.0...","[0.0053019347, -0.028574184, -0.013652737, -0...."
3,"Lastly, the advertisements that come up every ...",advertisements come time turn HD Kindle Fire a...,"[0.04314762, 0.056879345, -0.028308257, 0.0254...","[0.046416428, 0.05596209, -0.032902382, 0.0266..."
4,Convenient and use the library to download books,use library download books,"[0.052533846, 0.032760903, -0.013839496, 0.053...","[0.02805841, 0.010618144, -0.0151397865, 0.043..."
5,Should have done this a long time ago,have done time,"[0.00698723, -0.0040228614, -0.043284345, -0.0...","[-0.02697887, -0.037878744, -0.015118404, 0.00..."
6,The video is the best,video is,"[-0.025513561, -0.01481792, -0.024252268, 0.02...","[-0.03234921, -0.034854844, -0.030580219, 0.01..."
7,I don't bother watching the news on the televi...,do bother watching news television go Kindle Fire,"[0.029989352, 0.08369503, 0.0007162266, 0.0039...","[0.0080468375, 0.08008416, 0.0013907037, -0.03..."
8,Couldn't be happier with this,be,"[0.09142621, -0.02730996, -0.03349693, 0.02173...","[-0.04920172, -0.025933819, 0.04353434, 0.0044..."
9,Good reading device and easy to use,reading device use,"[0.04049769, -0.028381255, -0.0344359, 0.05621...","[0.029233314, 0.019851057, -0.043268554, 0.021..."


In [8]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
reviews['stemmed'] = reviews.text.map(lambda x: ' '.join([stemmer.stem(y) for y in x.split(' ')]))
reviews.stemmed.head()

0    i find that is not as easi to use as my regula...
1    when i am read the kindl it freez up (becaus y...
2    also you have to get out of it to brighten or ...
3    lastly, the advertis that come up everi time y...
4         conveni and use the librari to download book
Name: stemmed, dtype: object

In [12]:
reviews['stemmed'] = reviews.stemmed.apply(lambda x: re.sub(' +', ' ',x).strip())

In [15]:
reviews['stemmed'][4]

'conveni and use the librari to download book'

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from itertools import islice


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.2, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [69]:
cvec = CountVectorizer(stop_words='english', min_df=1, max_df=.2, ngram_range=(2,3))
cvec.fit(reviews.text)
len(cvec.vocabulary_)
#list(islice(cvec.vocabulary_.items(), 20))

22726

In [71]:
cvec_counts = cvec.transform(reviews.text)
print('sparse matrix shape:', cvec_counts.shape)
print('nonzero count:', cvec_counts.nnz)
print('sparsity: %.2f%%' % (100.0 * cvec_counts.nnz / (cvec_counts.shape[0] * cvec_counts.shape[1])))
occ = np.asarray(cvec_counts.sum(axis=0)).ravel().tolist()
counts_df = pd.DataFrame({'term': cvec.get_feature_names(), 'occurrences': occ})
#counts_df.sort_values(by='occurrences', ascending=False).head(20)

sparse matrix shape: (3355, 22726)
nonzero count: 25937
sparsity: 0.03%


In [75]:
transformer = TfidfTransformer()
transformed_weights = transformer.fit_transform(cvec_counts)
weights = np.asarray(transformed_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': cvec.get_feature_names(), 'weight': weights})
weights_df = weights_df.sort_values(by='weight', ascending=False).reset_index(drop=True)
weights_df.head(50)

,term,weight
0,easy use,0.006712
1,love kindle,0.003697
2,new kindle,0.003626
3,love new,0.003226
4,kindle hd,0.002948
5,year old,0.002825
6,battery life,0.002816
7,learning use,0.002584
8,love love,0.002315
9,love new kindle,0.002074


In [ ]:
def clean_review(x):

    words = {'love','loves',''}

resultwords  = [word for word in re.split("\W+",query) if word.lower() not in stopwords]
result = ' '.join(resultwords)
print(result)

In [ ]:
reviews['a1'] = reviews.text.apply(lambda x: )

In [95]:
cvec = CountVectorizer(stop_words='english', min_df=0.0005, max_df=.3, ngram_range=(2,2))
cvec.fit(reviews.text)
print('length',len(cvec.vocabulary_))
#list(islice(cvec.vocabulary_.items(), 20))
cvec_counts = cvec.transform(reviews.text)
print('sparse matrix shape:', cvec_counts.shape)
print('nonzero count:', cvec_counts.nnz)
print('sparsity: %.2f%%' % (100.0 * cvec_counts.nnz / (cvec_counts.shape[0] * cvec_counts.shape[1])))
occ = np.asarray(cvec_counts.sum(axis=0)).ravel().tolist()
counts_df = pd.DataFrame({'term': cvec.get_feature_names(), 'occurrences': occ})
#counts_df.sort_values(by='occurrences', ascending=False).head(20)
transformer = TfidfTransformer()
transformed_weights = transformer.fit_transform(cvec_counts)
weights = np.asarray(transformed_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': cvec.get_feature_names(), 'weight': weights})
weights_df = weights_df.sort_values(by='weight', ascending=False).reset_index(drop=True)
weights_df.head(50)

length 1280
sparse matrix shape: (3355, 1280)
nonzero count: 4149
sparsity: 0.10%


,term,weight
0,easy use,0.013272
1,battery life,0.007522
2,new kindle,0.006579
3,year old,0.006503
4,love new,0.005431
5,love kindle,0.005246
6,kindle hd,0.005154
7,original kindle,0.004690
8,learning use,0.004137
9,wi fi,0.004001


In [64]:
all_text = ''
for line in reviews.text.tolist():
    line = re.sub('(&#[0-9];.,?!)+', '',line)
    line = re.sub(' +', ' ',line)
    all_text = all_text + line

In [65]:
from rake_nltk import Rake

r = Rake(min_length=2, max_length=4) # Uses stopwords for english from NLTK, and all puntuation characters.

r.extract_keywords_from_text(all_text)

r.get_ranked_phrases()

['former asus memo padit',
 'pricefeeling heat towards top',
 'microsoft office functionsonline support',
 'everything preloaded isnt removable',
 'surface feels finehope heat',
 'book components ingeniously compressed',
 '&# 34 ;, ugh',
 'log ona little harder',
 'ipadno wonder apple leads',
 'creating filesruns extemely well',
 'sending documents via whispernet',
 'serve american express app',
 'experienced significant trouble connecting',
 'wireless connection fails authentication',
 'jokefor people needing guidance',
 'many repeats including 178',
 'extensivemy big thick fingers',
 'massive technological leap forward',
 'considering returning itbattery life',
 'edit google documents thoughneed',
 'devicemy kids absolutely ove',
 '4gbafter taking away 1gb',
 'lightest 7 &# 34',
 'oarrived 72 hours right',
 'liek music adn gamesi',
 'breezethe battery goes dead',
 'battery lifemine goes dead',
 'tec review people trip',
 'security featurekeyboard occasionally vanished',
 'absolutely 

In [ ]:
#sub=reviews.loc[:,['text','main']]
#sub.to_csv(r'/mnt/doc/School/USYD/Capstone/workspace/data_testing/output/sub-set/fireHD-tag.csv',sep='\t')

In [ ]:
# Preparing datasets
X1 = np.array(reviews['sentence_embedding'].tolist())
X2 = np.array(reviews['words_embedding'].tolist())
n_sentence = len(X1.tolist())

In [ ]:
####greedy
corr = np.inner(X1,X1)


In [ ]:
corr[corr<0]=0
corr[corr>0.99]=0

In [ ]:
result = np.where(corr == np.amax(corr))
result[0]

In [ ]:
corr[1193,1648]

In [ ]:
print(reviews['text'][1193])
print(reviews['text'][1648])

In [ ]:
corr_maxes = np.amax(corr,axis=1)
corr_mins = np.amin(corr,axis=1)
avg_corr = sum(sum(corr))/n_sentence/n_sentence
avg_max_corr = sum(corr_maxes) / n_sentence
avg_min_corr = sum(corr_mins) / n_sentence
print('avg max corr:{}, avg min corr:{}. avg corr:{}'.format(avg_max_corr, avg_min_corr, avg_corr))

In [ ]:
corr_min_of_max = np.amin(corr_maxes)
corr_min_of_max

In [ ]:
np.amax(corr)

In [ ]:
labels = np.zeros(shape=n_sentence, dtype=np.int8)
chain = np.zeros(shape=n_sentence, dtype=np.int8)

the_line = 0.85#0.9*avg_max_corr+0.1*avg_corr
the_line

In [ ]:
np.where(corr > the_line)[0].shape[0]/(n_sentence*n_sentence)

In [ ]:
pairs = np.where(corr > 0.85)

In [ ]:
my_i=4
print(reviews['text'][pairs[0][my_i]])
print(reviews['text'][pairs[1][my_i]])

In [ ]:
def find_close_pt_with_label(s): # s: the index of the sentence
    for i in range(0, n_sentence):
        if corr[s][i]>the_line and labels[i]!=0:
            return i,labels[i]
    return 0, 0

def group_labels(labels):
    clusters = []
    for i in range(-1,np.amax(labels)):
        index = np.where(labels == i)
        clusters.append(index[0])
    return clusters

In [ ]:
label_count = 1
current_label=0

for i in range(0,n_sentence):
    if np.amax(corr[i])<0.85:
        labels[i] = -1 
        continue
    #######check if we should skip to next ####################
    if labels[i]!= 0: # if i already has a label
        continue
    ######or need to find the current sentence a label########
    else:
        (index_, temp) = find_close_pt_with_label(i) # try to find i an existing label
        if temp!=0:
            labels[i] = temp
            chain[i] = 1+chain[index_]
            if chain[i]>1: #1: # does not allow 2+ chain
                labels[i] = label_count
                label_count = label_count+1
        else:
            labels[i] = label_count # else, assign a new label
            label_count = label_count+1
    ###### finished assigning the current sentence a label#########
    
    current_label = labels[i]
    ###### move on to spread the label  ###########################
    for j in range(i, n_sentence):
        if labels[j]!=0: # if j already has a label
            continue # skip
        if corr[i][j]>the_line: # if sentence i is similar to sentence j enough
            if labels[j]==0: # if sentence j doesn't have a label
                labels[j] = current_label
                

In [ ]:
clusters = group_labels(labels)

In [ ]:
len(clusters)

In [ ]:
clusters

In [ ]:
clusters[1].shape[0]

In [ ]:
loose = []
remove_group = []
for i in range(0,len(clusters)):
    if clusters[i].shape[0]<10:
        for j in range(0,clusters[i].shape[0]):
            loose.append(clusters[i][j])
        remove_group.append(i)
        
loose = np.array(loose)

In [ ]:
clusters = np.array(clusters)
clusters = np.delete(clusters, remove_group, 0)
clusters = clusters.tolist()
clusters.append(loose)

In [ ]:
clusters

In [ ]:
##############save##############
product_title='Fire HD'
params='greedy-cosine'
dir1 = save_results_dir(product_title,'similarity',params+'/sentence')
make_or_clean_dir(dir1)
for i in range(0,len(clusters)):    
    temp = reviews.loc[clusters[i].tolist(),['text']]
    file_name = str(i)+'.txt'
    np.savetxt(r''+dir1+file_name, temp.values, fmt='%s')

#################################

In [ ]:
pca = dc.PCA(n_components=10)
pca.fit(X1)
X1 = pca.transform(X1)

pca = dc.PCA(n_components=10)
pca.fit(X2)
X2 = pca.transform(X2)

n_sentence = len(X2.tolist())
import math
n_min_pts = math.ceil(n_sentence*0.01)

print('number of sentences:{}, min points:{}'.format(n_sentence,n_min_pts))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_similarity(labels, features, rotation):
    corr = np.inner(features, features)
    sns.set(font_scale=1.2)
    g = sns.heatmap(
      corr,
      xticklabels=labels,
      yticklabels=labels,
      vmin=0,
      vmax=1,
      cmap="YlOrRd")
    g.set_xticklabels(labels, rotation=rotation)
    g.set_title("Semantic Textual Similarity")

plot_similarity(np.array(reviews['text'].tolist())[0:50],np.array(reviews['sentence_embedding'].tolist())[0:50],90)

In [ ]:
label_s = np.array([])
label_w = np.array([])


def cluster_number(labels):
    return (len(set(labels)) - (1 if -1 in labels else 0))



In [ ]:
# Clustering
time1 = time.time()
db_s = DBSCAN(eps=0.246, min_samples=n_min_pts, metric='canberra').fit(X1)
print('time for clustering sentence embeddings:', (time.time()-time1))
time1 = time.time()
db_w = DBSCAN(eps=0.246, min_samples=n_min_pts, metric='canberra').fit(X2)
print('time for clustering words embeddings:',(time.time()-time1))
# Get labels
labels_s = db_s.labels_.tolist()
labels_w = db_w.labels_.tolist()
reviews['label_db_sentence'] = labels_s
reviews['label_db_word'] = labels_w
n_sentence= cluster_number(labels_s)
n_words= cluster_number(labels_w)

In [ ]:
print('total sentences:',len(list(labels_s)))
print('clusters for sentences:',n_sentence, 'noise:',list(labels_s).count(-1))
print('clusters for words:',n_words, 'noise:',list(labels_w).count(-1))

In [ ]:
def save_results_dir(product_name, method, params):
    dir_pre = '/home/betty35/桌面/Capstone/workspace/data_testing/output/'
    dir_ = dir_pre+product_name+'/'+method+'/'+params+'/'
    return dir_


def make_dir(dir_):
    if not os.path.exists(dir_):
        try:
            os.makedirs(dir_)
        except FileExistsError:
            pass

def clean_dir(dir_):
    for file_ in os.listdir(dir_):
        file_dir = os.path.join(dir_, file_)
        try:
            if os.path.isfile(file_dir):
                os.unlink(file_dir)
            elif os.path.isdir(file_dir): shutil.rmtree(file_dir)
        except Exception as e:
            print(e)

def make_or_clean_dir(dir_):
    make_dir(dir_)
    clean_dir(dir_)
    

In [ ]:
product_title='Fire HD'
params='0.27-PCA10'
dir1 = save_results_dir(product_title,'dbscan',params+'/sentence')
make_or_clean_dir(dir1)
for i in range(0,n_sentence):    
    temp = reviews.loc[reviews['label_db_sentence']==i,['text']]
    file_name = str(i)+'.txt'
    np.savetxt(r''+dir1+file_name, temp.values, fmt='%s')

noise_s = temp = reviews.loc[reviews['label_db_sentence']==-1,['text']] 
np.savetxt(r''+dir1+'noise.txt', temp.values, fmt='%s')

dir1 = save_results_dir(product_title,'dbscan',params+'/word')
make_or_clean_dir(dir1)
for i in range(0,n_words):
    temp = reviews.loc[reviews['label_db_word']==i,['text']]
    file_name = str(i)+'.txt'
    np.savetxt(r''+dir1+file_name, temp.values, fmt='%s')
noise_s = temp = reviews.loc[reviews['label_db_word']==-1,['text']] 
np.savetxt(r''+dir1+'noise.txt', temp.values, fmt='%s')

In [ ]:
def cal_distance(X250):
    